In [ ]:
import getpass
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [ ]:
from openpyxl import load_workbook
wb = load_workbook(filename='Organizations_module.xlsx')
ws = wb['output']

source_data = list()
for num, row in enumerate(ws.rows):
    if num == 0:
        headers = [i.value for i in row]
    else:
        source_data.append(dict(zip(headers, [i.value for i in row])))

In [ ]:
driver = webdriver.Firefox()

In [ ]:
def open_site():
    driver.get("https://lsu-erm.bc.sirsidynix.net/auth/?service=/organizations/")

In [ ]:
open_site()

In [ ]:
def open_resultspage_by_firstletter(firstletter):
    try:
        shortcut = driver.find_element_by_xpath('//span[@id="span_letter_{}"]'.format(firstletter))
        shortcut.click()
    except NoSuchElementException:
        # use false to indicate no recorded titles begin with firstletter
        return False

In [ ]:
def open_resultspage_by_partial_word(partial_word):
    input_box = driver.find_element_by_xpath('//')

In [ ]:
def org_in_dataTable(org):
    try:
        link = driver.find_element_by_xpath('//table[@class="dataTable"]/tbody/tr/td/a[contains(text(), "{}")]'.format(org))
        return link
    except NoSuchElementException:
        # use false to indicate no recorded org with that title
        return False

In [ ]:
founds = list()
not_founds = list()

for item in source_data:
    organization = item['Publisher']
    open_resultspage_by_firstletter(organization[0])
    time.sleep(2)
    match = org_in_dataTable(organization)
    if match:
        founds.append((organization, match.text))
    else:
        not_founds.append(organization)
        
# there are ~10 orgs already entered in some degree.  Easier to do them manually.

In [ ]:
def go_to_homepage():
    link = driver.find_element_by_xpath('//div[@class="main-menu-link active"]/span[contains(text(), "Home")]')
    link.click()

In [ ]:
def enter_org_page(org):
    link = driver.find_element_by_xpath('//table[@class="dataTable"]/tbody/tr/td/a[contains(text(), "{}")]'.format(org))
    link.click()

In [ ]:
def make_new_org(item):
    new_org_link = driver.find_element_by_xpath('//div[@class="main-menu-link"]/span[contains(text(), "New Organization")]')
    new_org_link.click()
    time.sleep(2)
    name_field = driver.find_element_by_xpath('//input[@id="organizationName"]')
    name_field.send_keys(item['vendor_publisher'])
    parent_field = driver.find_element_by_xpath('//input[@id="parentOrganization"]')
    parent_field.send_keys(item['parent_name'])
    url_field = driver.find_element_by_xpath('//input[@id="companyURL"]')
    url_field.clear()
    url_field.send_keys(item['url'])
    submit_button = driver.find_element_by_xpath('//input[@id="submitOrganizationChanges"]')
    submit_button.click()

In [ ]:
def add_contact_info(item):
    contact_button = driver.find_element_by_xpath('//a[@class="showContacts"]')
    contact_button.click()
    time.sleep(2)
    add_contact_link = driver.find_element_by_xpath('//a[contains(text(), "add contact")]')
    add_contact_link.click()
    time.sleep(2)
    email_field = driver.find_element_by_xpath('//input[@id="emailAddress"]')
    email_field.send_keys(item['contact_email'])
    phone_field = driver.find_element_by_xpath('//input[@id="phoneNumber"]')
    phone_field.send_keys(item['contact_phone'])
    support_toggle = driver.find_element_by_xpath('//input[@id="4"]')
    support_toggle.click()
    submit_button = driver.find_element_by_xpath('//input[@id="submitContactForm"]')
    submit_button.click()

In [ ]:
for item in source_data:
    print(item['vendor_publisher'])
    if item['vendor_publisher'] in not_founds:
        make_new_org(item)
        add_contact_info(item)
        time.sleep(2)
        break
    else:
        print('{} not in "not_founds"'.format(item['vendor_publisher']))

In [ ]:
source_data[0]